In [4]:
import pandas as pd

### Variable Type

<img src="assets/variables_type.png" alt="Variables Type" width="1000">

### Unfolding type

<img src="assets/unfolding_type.png" alt="Unfolding Type" width="1000">

In [16]:
def read_owu(file):
	root_path = 'dataset/datahow_2020/insilico_data'
	data = pd.read_excel(f'{root_path}/{file}.xlsx')
	col_names = ["run", "timesteps", "X:VCD", "X:Glc", "X:Gln", "X:NH4", "X:Lac", "X:Titer", "W:pH", "W:Temp", "F:Feed_Glc", "F:Feed_Gln",]
	owu_df = data.copy()
	owu_df.columns = col_names
	owu_df['time'] = (owu_df.timesteps / 24).astype(int)
	owu_df.set_index(['run', 'time'], inplace=True)
	return owu_df

In [17]:
owu = read_owu('rawdata')

In [18]:
owu.head(20)

timesteps     X:VCD       X:Glc     X:Gln     X:NH4       X:Lac  \
run time                                                                    
1   0             0  0.400000   60.000000  3.000000  0.100000    0.100000   
    1            24  1.015551   82.789425  2.071044  0.945923    5.292220   
    2            48  2.063143   73.895885  1.504454  3.228866   15.598221   
    3            72  2.735150  106.519232  0.374085  3.251472   33.851222   
    4            96  4.106711  102.865067  0.174484  4.377329   43.038763   
    5           120  4.723016   88.211028  0.000000  5.223006   36.496690   
    6           144  4.932028  168.364784  0.000000  5.769891   74.991427   
    7           168  4.827316  133.857140  0.039018  4.991701  105.321299   
    8           192  5.745480  199.244957  0.136537  5.620637  125.286756   
    9           216  4.781973  234.815768  0.000000  5.993455  121.762157   
    10          240  2.934622  137.680830  0.103169  3.259237  119.732154   
    11          264  3.543456  184.345047  0.054787  4.760620  102.848334   
    12          288  2.403908  162.281839  0.000000  4.463759  136.020096   
    13          312  2.013994  139.770075  0.000000  3.140056  123.428555   
    14          336  1.682560  183.677813  0.103853  5.023750  120.646184   
2   0             0  0.400000   20.000000  8.000000  0.100000    0.100000   
    1            24  0.953393   17.539893  6.769361  2.889616    4.587876   
    2            48  1.786357    9.315830  5.623043  4.010230   15.081016   
    3            72  1.991316    1.101207  2.737111  6.051084   17.550484   
    4            96  3.897274   17.605891  2.633381  7.972448   43.170690   

             X:Titer  W:pH  W:Temp  F:Feed_Glc  F:Feed_Gln  
run time                                                    
1   0       0.000000   6.9    37.5         0.0         0.0  
    1       8.891501   6.9    37.5        30.0         1.0  
    2      23.479590   6.9    37.5        30.0         1.0  
    3      73.414915   6.9    37.5        30.0         1.0  
    4     150.955189   6.9    37.5        30.0         1.0  
    5     189.689869   6.9    37.5        30.0         1.0  
    6     346.178775   7.0    37.5        30.0         1.0  
    7     348.716061   7.0    37.5        30.0         1.0  
    8     322.798230   7.0    37.5         0.0         0.0  
    9     430.402555   7.0    37.5         0.0         0.0  
    10    433.841473   7.0    37.5         0.0         0.0  
    11    446.706953   7.0    37.5         0.0         0.0  
    12    395.599644   7.0    37.5         0.0         0.0  
    13    453.569183   7.0    37.5         0.0         0.0  
    14    438.802228   7.0    37.0         0.0         0.0  
2   0       0.000000   7.1    36.0         0.0         0.0  
    1       4.177204   7.1    36.0         0.0         0.0  
    2      35.912374   7.1    36.0         0.0         0.0  
    3      52.764046   7.1    36.0        30.0         1.0  
    4     118.114401   7.1    36.0        30.0         1.0

In [19]:
def generate_bwu(owu):
    owu = owu.drop(["timesteps"],axis=1)
    # Input: multiindex OWU
    # Output: singleindex BWU
    for run_ix,run in owu.groupby("run"):
        if run_ix == 1: 
            bwuindex = run.unstack(level=1)
        else:
            bwuindex = pd.concat([bwuindex, run.unstack(level=1)])
    bwu_columns = [str(bwuindex.columns.get_level_values(0)[i])+str(":")+str(bwuindex.columns.get_level_values(1)[i]) 
                   for i in range(len(bwuindex.columns.get_level_values(0)))]
    bwu = pd.DataFrame(bwuindex.to_numpy(), columns=bwu_columns)
    
    return bwu

In [20]:
bwu = generate_bwu(owu)
bwu.head()

,X:VCD:0,X:VCD:1,X:VCD:2,X:VCD:3,X:VCD:4,X:VCD:5,X:VCD:6,X:VCD:7,X:VCD:8,X:VCD:9,...,F:Feed_Gln:5,F:Feed_Gln:6,F:Feed_Gln:7,F:Feed_Gln:8,F:Feed_Gln:9,F:Feed_Gln:10,F:Feed_Gln:11,F:Feed_Gln:12,F:Feed_Gln:13,F:Feed_Gln:14
0,0.4,1.015551,2.063143,2.735150,4.106711,4.723016,4.932028,4.827316,5.745480,4.781973,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.4,0.953393,1.786357,1.991316,3.897274,4.040470,3.695521,3.945214,4.182197,3.642935,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.4,1.914412,3.033244,2.429069,4.003605,5.821906,6.183043,5.095282,4.591090,3.885602,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.2,0.485637,1.470946,2.234323,4.404896,4.952949,5.638315,4.904156,5.515848,4.060728,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,0.2,0.701393,1.722991,1.884669,2.493436,3.742112,4.650736,4.441868,4.666359,5.576590,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def generate_y(bwu, return_aggr=False):
    # Input: singleindex BWU
    # Output: singleindex BWU having only target
    titer_column = [c for c in bwu.columns if c.startswith("X:Titer")]
    targets = pd.DataFrame(columns=["Y:Titer", "Y:Aggr"], index=bwu.index)

    # iterate through experiments
    for j in list(bwu.index):
        x_titer = bwu.loc[j, titer_column]
        x_prod = [0]
        x_aggr = [0]
        k_aggr = 10**-7
        for i in range(len(x_titer)):
            if i == 0:
                continue
            xt_titer = x_titer[i]
            dt_titer = x_titer[i] - x_titer[i - 1]
            x_prod.append(xt_titer)
            x_aggr.append(k_aggr * (xt_titer**2))

            dt_aggr = x_aggr[i] - x_aggr[i - 1]
            dt_prod = dt_titer - 2 * dt_aggr
            dt_aggr = k_aggr * (x_prod[i - 1] + dt_prod) ** 2

            x_aggr[i] = x_aggr[i - 1] + dt_aggr
            x_prod[i] = x_prod[i - 1] + dt_prod
        y_prod = x_prod[-1]
        y_aggr = x_aggr[-1]

        targets.loc[j, "Y:Titer"] = y_prod
        targets.loc[j, "Y:Aggr"] = y_aggr
    if return_aggr:
        target = targets["Y:Aggr"]
    else:
        target = targets["Y:Titer"]

    return pd.DataFrame(target)

In [22]:
tar = generate_y(bwu, return_aggr=False)

In [23]:
tar

,Y:Titer
0,439.66522
1,526.159742
2,1047.518676
3,843.761678
4,375.424487
...,...
95,818.250952
96,626.48565
97,702.698925
98,1120.810631
